In [ ]:
import pandas as pd
df = pd.read_json("/Users/cameliamazouz/Documents/M2/machine_learning/multinli_1.0/multinli_1.0_train.jsonl",lines=True)

In [ ]:
df.columns

#### lemmatisation 


In [ ]:
# Fonctions de lemmatisation (NLTK)
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

def ensure_nltk_resources():
    # Tentative de téléchargement silencieux des ressources utiles
    for pkg in ["punkt", "wordnet", "omw-1.4", "averaged_perceptron_tagger", "averaged_perceptron_tagger_eng"]:
        try:
            nltk.download(pkg, quiet=True)
        except Exception:
            pass

def to_wordnet_pos(treebank_tag: str):
    # Map POS tag PennTreebank vers WordNet POS
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    if treebank_tag.startswith('V'):
        return wordnet.VERB
    if treebank_tag.startswith('N'):
        return wordnet.NOUN
    if treebank_tag.startswith('R'):
        return wordnet.ADV
    return wordnet.NOUN  # défaut raisonnable

def lemmatize_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    tokens = word_tokenize(text)
    tags = pos_tag(tokens)
    wnl = WordNetLemmatizer()
    lemmas = [wnl.lemmatize(tok, pos=to_wordnet_pos(tag)) for tok, tag in tags]
    return " ".join(lemmas)

def lemmatize_columns(df, columns=("sentence1", "sentence2")):
    ensure_nltk_resources()
    df_out = df.copy()
    for col in columns:
        if col in df_out.columns:
            df_out[f"{col}_lemma"] = df_out[col].astype(str).apply(lemmatize_text)
    return df_out

In [ ]:
# Appliquer la lemmatisation à df et sauvegarder
# Assurez-vous d'avoir exécuté les cellules qui définissent df
df_lemma = lemmatize_columns(df, columns=("sentence1", "sentence2"))
print("Colonnes disponibles:", df_lemma.columns.tolist())
print("Aperçu lemmatisé:")
display(df_lemma[["sentence1", "sentence1_lemma", "sentence2", "sentence2_lemma" ]].head())

# Sauvegarde
import os
os.makedirs("artifacts", exist_ok=True)
out_lemma_csv = "artifacts/df_lemmatized.csv"
df_lemma.to_csv(out_lemma_csv, index=False)
print(f"Sauvegardé: {out_lemma_csv} (shape: {df_lemma.shape})")

In [ ]:
# df["genre_id"] = df["genre"].astype("category").cat.codes
# df["label_id"] = df["gold_label"].astype("category").cat.codes

# # Vérification
# print(df[["genre", "genre_id", "gold_label", "label_id"]].head())

#### Vecteurs

In [ ]:
label_map = {"entailment": 0, "neutral": 1, "contradiction": 2}
df["label_id"] = df["gold_label"].map(label_map)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Nettoyage rapide (les NaNs font planter le vectorizer)
df["sentence1"] = df["sentence1"].fillna("")
df["sentence2"] = df["sentence2"].fillna("")

# 2. Création du Vectorizer
# On peut ajouter stop_words='english' pour virer les "the", "is", "a"...
vectorizer = CountVectorizer(stop_words='english', max_features=1000) 

# 3. Apprentissage du vocabulaire sur TOUT le texte (S1 + S2)
# C'est crucial pour que la colonne 42 corresponde au mot "apple" dans les deux vecteurs
all_text = pd.concat([df["sentence1"], df["sentence2"]])
vectorizer.fit(all_text)

# 4. Transformation en vecteurs de fréquence
# X1 = Fréquence des mots dans sentence1
# X2 = Fréquence des mots dans sentence2 (en utilisant le même dictionnaire que S1)
X1 = vectorizer.transform(df["sentence1"])
X2 = vectorizer.transform(df["sentence2"])


In [ ]:
import pandas as pd

# On choisit l'index que tu voulais (1)
idx = 1

# 1. Récupérer le dictionnaire (la liste des mots dans l'ordre des colonnes)
vocabulaire = vectorizer.get_feature_names_out()

# 2. Récupérer les comptes pour la ligne choisie
# .flatten() permet d'aplatir le tableau (convertir [[0,1]] en [0,1])
compte_s1 = X1[idx].toarray().flatten()
compte_s2 = X2[idx].toarray().flatten()

# 3. Créer un tableau propre pour l'affichage
df_visu = pd.DataFrame({
    'Mot': vocabulaire,
    'S1 (Freq)': compte_s1,
    'S2 (Freq)': compte_s2
})

# 4. FILTRAGE : On n'affiche que les mots présents dans au moins l'une des phrases
# (Sinon on va afficher 990 lignes de zéros)
mask = (df_visu['S1 (Freq)'] > 0) | (df_visu['S2 (Freq)'] > 0)
df_resultat = df_visu[mask].sort_values(by='S1 (Freq)', ascending=False)

# --- AFFICHAGE ---
print(f"--- Analyse de l'index {idx} ---\n")
print(f"Phrase 1 : \"{df['sentence1'].iloc[idx]}\"")
print(f"Phrase 2 : \"{df['sentence2'].iloc[idx]}\"\n")
print("Mots comptés (intersection avec le vocabulaire connu) :")
print(df_resultat.to_string(index=False))

In [ ]:
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# On colle les matrices horizontalement : [Features S1] + [Features S2]
X_combined = hstack([X1, X2])
y = df["label_id"]

print(f"Split des données (10% test sur {len(df)} lignes)...")
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y, test_size=0.10, random_state=42
)

# 6. ENTRAÎNEMENT (Régression Logistique)
print("Entraînement du modèle...")
model = LogisticRegression(max_iter=1000) # max_iter augmenté pour la convergence
model.fit(X_train, y_train)

# 7. ÉVALUATION
predictions = model.predict(X_test)
acc = accuracy_score(y_test, predictions)

print(f"\n--- RÉSULTATS ---")
print(f"Accuracy (Précision globale) : {acc:.4f} ({acc*100:.2f}%)")

# Détail par classe
target_names = ["Entailment (0)", "Neutral (1)", "Contradiction (2)"]
print("\nDétail par classe :")
print(classification_report(y_test, predictions, target_names=target_names))